In [1]:
# 필요한 library 를 import
from selenium import webdriver
import time
from bs4 import BeautifulSoup as bs

In [2]:
def NaverCafeCrawl(keyname):
    # driver 불러오기 (chromedriver.exe를 다운받은 후 그 파일경로 넣어주기)
    driver = webdriver.Chrome("C:/Users/Joobe/Downloads/chromedriver.exe") # 빈 화면의 크롬을 열어준다. 
    driver.get("https://search.naver.com/search.naver?where=article&sm=tab_jum&query=") # driver.get(열고 싶은 url)
    keyword = driver.find_element_by_name("query") # 입력 키워드의 태그를 query로 지정
    keyword.send_keys(keyname) # 검색하고자 하는 키워드 넣어주기.
    # 검색(돋보기)버튼 누르기
    driver.find_element_by_css_selector("#nx_search_form > fieldset > button > span").click() 
    
    # 네이버는 최대 100페이지, 페이지당 10건해서 총 1천 개의 글만 지원하므로 루프를 100번 돌게 해줌.
    for page in range(100):
        for i in range(1,11,1):
            # 검색된 카페 글(한 페이지에 10개)에 차례대로 들어가기
            driver.find_element_by_xpath('//*[@id="elThumbnailResultArea"]/li['+ str(i) +']/dl/dt/a').click()
            # 가장 최근 열린 탭으로 전환. 해주지 않으면 기존 네이버 검색창의 html만 얻을 수 있기 때문 
            driver.switch_to.window(driver.window_handles[-1])
            # 로딩 기다리기
            time.sleep(5)
    
            html = driver.page_source
            soup = bs(html, "html.parser")
    
            # 카페 글은 iframe 형태이며, iframe 은 html을 숨겨놓은 형태임.
            # 따라서 진짜 html이 있는 곳으로 접속이 필요함.
            # 네이버 카페의 경우, cafe_main 이라는 frame에 진짜 html이 숨어 있음.
            driver.switch_to.frame("cafe_main") # 프레임 전환해주기
            soups = bs(driver.page_source, "html.parser")
    
            # 카페 글의 제목 추출
            title = soups.select("div.tit-box span.b")
            title_data = title[0].text
    
            # 카페 글의 본문 추출
            para = soups.find_all("div",class_="tbody m-tcol-c")
            para_data = para[0].text
    
            # 파일에 쓰기 (NaverCafe_검색어.txt로 저장됨)
            f = open("C:/Users/Joobe/Downloads/NaverCafe_{0:}.txt".format(keyname), "a", encoding="utf-8")
            f.write("제목:"+ title_data)
            f.write(para_data)
    
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
        # 한 페이지 내 10개 글을 다 읽어들였으니, 다음 페이지로 넘어가기
        driver.find_element_by_css_selector("#main_pack > div.paging > a.next").click()

In [3]:
# NaverCafeCrawl("검색어")
NaverCafeCrawl("서귀포")